In [23]:
import pandas as pd
import numpy as np

In [3]:
class Option_pricer:
    def __init__(self, S0, r, q, T, model, params):
        self.S0 = S0
        self.r = r
        self.q = q
        self.T = T
        self.model = model
        self.params = params

    
#Characteristic function
  
    def CF(self, u):
        S0, r, q, T = self.S0, self.r, self.q, self.T
        u = np.array(u, dtype=complex)

        if self.model == 'GBM':
            sigma = self.params[0]
            mu = np.log(S0) + (r - q - 0.5 * sigma**2) * T
            return np.exp(1j * u * mu - 0.5 * (sigma * np.sqrt(T) * u)**2)

        elif self.model == 'Heston':
            kappa, theta, sigma, rho, v0= self.params
            tmp = kappa - 1j * rho * sigma * u
            g = np.sqrt((sigma**2) * (u**2 + 1j*u) + tmp**2)
            pow1 = 2 * kappa * theta / sigma**2
            term1 = (kappa * theta * T * tmp) / sigma**2 + 1j * u * (np.log(S0) + (r - q)*T)
            term2 = pow1 * np.log(np.cosh(g*T/2) + (tmp/g)*np.sinh(g*T/2))
            term3 = ((u**2 + 1j*u) * v0) / (g/np.tanh(g*T/2) + tmp)
            return np.exp(term1 - term2 - term3)

        elif self.model == 'VG':
            sigma, nu, theta = self.params
            if nu == 0:
                mu = np.log(S0) + (r - q - theta - 0.5*sigma**2) * T
                return np.exp(1j * u * mu + (1j*theta*u - 0.5*sigma**2*u**2)*T)
            else:
                mu = np.log(S0) + (r - q + np.log(1 - theta*nu - 0.5*sigma**2*nu)/nu) * T
                return np.exp(1j * u * mu) * (1 - 1j*theta*nu*u + 0.5*nu*sigma**2*u**2) ** (-T/nu)

 
# FFT pricing
    
    def price_fft(self, K, alpha=1.5, eta=0.25, n=12):
        N = 2**n
        df = np.exp(-self.r*self.T)
        lda = 2*np.pi/(N*eta)
        beta = np.log(K)

        nu = np.arange(N)*eta
        denom = (alpha + 1j*nu)*(alpha + 1 + 1j*nu)
        psi = self.CF(nu - 1j*(alpha+1)) / denom

        w = np.full(N, eta)
        w[0] = eta/2
        x = df * psi * w * np.exp(-1j*beta*nu)

        y = np.fft.fft(x)
        km = beta + np.arange(N)*lda
        cT = np.exp(-alpha*km) * np.real(y)/np.pi

        return km, cT

   
# Direct integration pricing
    
    def price_integral(self, K, alpha=1.5, eta=0.25, n=12):
        N = 2**n
        df = np.exp(-self.r*self.T)
        k = np.log(K)

        nu = np.arange(N)*eta
        denom = (alpha + 1j*nu)*(alpha + 1 + 1j*nu)
        psi = df * self.CF(nu - 1j*(alpha+1)) / denom

        w = np.full(N, eta)
        w[0] = eta/2
        integral = np.sum(np.exp(-1j*nu*k) * psi * w)
        return np.real(np.exp(-alpha*k) * integral / np.pi)

    
# Pricing put using put-call parity
    
    def put_price(self, C, K):
        S0, r, q, T = self.S0, self.r, self.q, self.T
        return C - S0 * np.exp(-q*T) + K * np.exp(-r*T)


In [10]:


S0 = 100
K = 80
r = 0.05
q = 0.01
T = 365 / 365
alpha = 1.5
eta = 0.25
n = 12

params = [0.30]

pricer = Option_pricer(S0, r, q, T, 'GBM', params)

km, prices_fft = pricer.price_fft(K, alpha, eta, n)
call_fft_price = prices_fft[0]
put_fft_price = pricer.put_price(call_fft_price, K)

call_int_price = pricer.price_integral(K, alpha, eta, n)
put_int_price = pricer.put_price(call_int_price, K)

print(f'                spot price : {S0:.4f} ')
print('=====================================')
print(f'                     Model : {pricer.model}')
print(f'      Call Price using FFT : {call_fft_price:.4f}')
print(f' Call Price using Integral : {call_int_price:.4f}')
print('----------------------------')
print('Put Price')
print('----------------------------')
print(f'       Put Price using FFT : {put_fft_price:.4f}')
print(f'  Put Price using Integral : {put_int_price:.4f}')

                spot price : 100.0000 
                     Model : GBM
      Call Price using FFT : 25.6146
 Call Price using Integral : 25.6146
----------------------------
Put Price
----------------------------
       Put Price using FFT : 2.7080
  Put Price using Integral : 2.7080


In [18]:
params = [2.0, 0.09, 0.40, -0.70, 0.14]

pricer = Option_pricer(S0, r, q, T, 'Heston', params)

km, prices_fft = pricer.price_fft(K, alpha, eta, n)
call_fft_price = prices_fft[0]
put_fft_price = pricer.put_price(call_fft_price, K)

call_int_price = pricer.price_integral(K, alpha, eta, n)
put_int_price = pricer.put_price(call_int_price, K)

print(f'                spot price : {S0:.4f} ')
print('=====================================')
print(f'                     Model : {pricer.model}')
print(f'      Call Price using FFT : {call_fft_price:.4f}')
print(f' Call Price using Integral : {call_int_price:.4f}')
print('----------------------------')
print('Put Price')
print('----------------------------')
print(f'       Put Price using FFT : {put_fft_price:.4f}')
print(f'  Put Price using Integral : {put_int_price:.4f}')

                spot price : 100.0000 
                     Model : Heston
      Call Price using FFT : 26.9027
 Call Price using Integral : 26.9027
----------------------------
Put Price
----------------------------
       Put Price using FFT : 3.9961
  Put Price using Integral : 3.9961


In [16]:
params = [0.30, 0.5, 0.070]

pricer = Option_pricer(S0, r, q, T, 'VG', params)

km, prices_fft = pricer.price_fft(K, alpha, eta, n)
call_fft_price = prices_fft[0]
put_fft_price = pricer.put_price(call_fft_price, K)

call_int_price = pricer.price_integral(K, alpha, eta, n)
put_int_price = pricer.put_price(call_int_price, K)

print(f'                spot price : {S0:.4f} ')
print('=====================================')
print(f'                     Model : {pricer.model}')
print(f'      Call Price using FFT : {call_fft_price:.4f}')
print(f' Call Price using Integral : {call_int_price:.4f}')
print('----------------------------')
print('Put Price')
print('----------------------------')
print(f'       Put Price using FFT : {put_fft_price:.4f}')
print(f'  Put Price using Integral : {put_int_price:.4f}')

                spot price : 100.0000 
                     Model : VG
      Call Price using FFT : 25.3256
 Call Price using Integral : 25.3256
----------------------------
Put Price
----------------------------
       Put Price using FFT : 2.4189
  Put Price using Integral : 2.4189
